In [165]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import re
import unicodedata
import os
import codecs
import nltk
import multiprocessing

from nltk.corpus import *
from collections import *
from gensim import *
from gensim.models import *
from sklearn import *
from sklearn.metrics.pairwise import *
from nltk.stem import *
with open("stopwords.txt") as f:
    stopwords = [word for line in f for word in line.split()]
df = pd.read_csv('data/p4kreviews.csv', skipinitialspace=True)

lemmatizer = WordNetLemmatizer()

In [159]:
def clean_text(text):
    temp = text
    temp=re.sub('\S+[\'\’\‘]\S+','',(temp))
    temp=re.sub('\w*\d\S*','',(temp))
    temp=re.sub('(?<!^|$)(?<!([(\.)(\!)(\?)(\“)]\s))([A-Z]\S+)','',(temp))
    temp=re.sub('[^A-z\s\-\–\&]',' ',(temp))
    temp=re.sub('\su\ss\s',' U.S. ',(temp))
    temp=re.sub('\[#.+]\|+','',(temp))
    temp = temp.lower()
    return temp

In [168]:
reviews = [str(re.sub('—',' ',str(df.loc[i].review))).decode('unicode_escape').encode('ascii','ignore') for i in list(range(len(df.loc[:])))]
total_text = ' '.join([reviews[i] for i in list(range(len(reviews)))])
cleaned_orig_reviews = [clean_text(text) for text in reviews]

In [108]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'
    elif tag.startswith('V'):
        return 'v'
    elif tag.startswith('N'):
        return 'n'
    elif tag.startswith('R'):
        return 'r'
    else:
        return 'n'

In [109]:
def remove_stop(m):
    return '' if m.group() in stopwords else m.group()

In [152]:
reviews = [ re.sub(r'\w+(\-|\—|\.|\&|\’)?(\w+)?', remove_stop, review) for review in reviews]
reviews = [ re.sub(r'\-+\s', ' ', review) for review in reviews]
total_text = ' '.join([reviews[i] for i in list(range(len(reviews)))])
total_words = len(total_text)
vocab_counter = Counter(total_text.split(' '))


sentences = [re.split('[\.\!\?]\s',re.sub('[^A-z\.\!\?\-\&]+',' ',review.lower())) for review in reviews]
 tempences = []
 for i in list(range(len(sentences))):
     tempences.append(list(filter(None, [o.split() for o in sentences[i]])))
 sentences = tempences

rawdata = {'doc_id':[],'review':[]}
for i in list(range(len(reviews))):
    tempreview = str(reviews[i])
    tempreview = clean_text(tempreview)
    tempreview = str(' '.join([lemmatizer.lemmatize(tag[0], get_wordnet_pos(tag[1])) for tag in pos_tag(tempreview.split())]))
    rawdata['doc_id'].append(i)
    rawdata['review'].append(tempreview)
rawdata['review'] = [ re.sub(r'\w+(\-|\—|\.|\&|\’)?(\w+)?', remove_stop, review) for review in rawdata['review']]

df2 = pd.DataFrame(rawdata,columns=['doc_id','review'])

df2.to_csv('reviews.csv')

metadata = df.drop('review',axis=1)

metadata['doc_id'] = rawdata['doc_id']

metadata.to_csv('metadata.csv')

documents = rawdata['review']
texts = [[word for word in document.lower().split() if word not in stopwords]
          for document in documents]
frequency = defaultdict(int)
for text in texts:
    for token in text:
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
    for text in texts]
dictionary = corpora.Dictionary(texts)
dictionary.save('reviewsDict.dict')
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('reviewsDict.mm', corpus)

documents = cleaned_orig_reviews
texts = [[word for word in document.lower().split() if word not in stopwords]
          for document in documents]
frequency = defaultdict(int)
for text in texts:
    for token in text:
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
    for text in texts]
dictionary = corpora.Dictionary(texts)
dictionary.save('models/orig_text.dict')
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('models/orig_text.mm', corpus)

stopwords= set(stopwords.words('english'))
stopwords = [i.encode('ascii','ignore') for i in list(stopwords)]
stopwords = set(stopwords)
stopwords |= set(['music','album','albums','band','bands','artist','artists','song','songs',
                  'track','tracks','time','length','\\x','year','years','would','something',
                  'time','often','yet','another','title','seems','words','well','really','right',
                  'thing','enough','got','know','say','released','people',
                  'best','new','reissue','since','less','go','label'])
counter = Counter(total_text.lower().split(' '))
ignore = list(stopwords)
for word in list(counter):
    if word in ignore:
        del counter[word]
stopwords |= set([counter.most_common(80)[i][0] for i in list(range(80))])
nonstop = ['pop','—','guitar', 'rock', 'indie', 'black', 'debut', 'indie', 'label', 'solo', 'vocal', 'vocals', 'voice', 'love', 'world', 'live', 'lyrics' ]
for word in nonstop:
    if(word in stopwords):
        stopwords.remove(word)
temp = [word.replace('\'','') for word in stopwords] 
stopwords |= set(temp)
temp = [word.replace('\'','’') for word in stopwords] 
stopwords |= set(temp)
temp = [word.title() for word in stopwords] 
stopwords |= set(temp)